In [1]:
pip install pandas

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 12.4 MB 654 kB/s eta 0:00:01
     |████████████████████████████████| 345 kB 687 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 9.1 MB 572 kB/s eta 0:00:01
     |████████████████████████████████| 736 kB 517 kB/s eta 0:00:01
     |████████████████████████████████| 776 kB 645 kB/s eta 0:00:01
     |████████████████████████████████| 401 kB 669 kB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 638 kB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 652 kB/s eta 0:00:01
     |████████████████████████████████| 316 kB 653 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-learn

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 11.1 MB 454 kB/s eta 0:00:01
     |████████████████████████████████| 301 kB 506 kB/s eta 0:00:01
     |████████████████████████████████| 34.5 MB 466 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch  # 导入PyTorch库
import numpy as np  # 导入NumPy库
import pandas as pd  # 导入Pandas库
from transformers import BertTokenizer, BertModel  # 从Transformers库中导入BertTokenizer和BertModel
from torch import nn  # 导入PyTorch的神经网络模块
from torch.optim import Adam  # 导入PyTorch的Adam优化器
from tqdm import tqdm  # 导入tqdm库用于进度条显示
import gc  # 导入垃圾回收模块
import os  # 导入操作系统接口模块

# 清理GPU内存
def free_gpu_memory():
    gc.collect()  # 垃圾回收
    torch.cuda.empty_cache()  # 清空CUDA缓存

free_gpu_memory()  # 调用清理GPU内存的函数

tokenizer = BertTokenizer.from_pretrained('model')  # 从预训练模型加载BertTokenizer

# 定义标签字典
labels = {'white': 0, 'sex': 1, 'scam': 2, 'gamble': 3, 'black': 4}

# 定义滑动窗口分词器函数
def sliding_window_tokenizer(text, tokenizer, max_length=128, stride=64):
    encoding = tokenizer(text, return_tensors='pt', truncation=False)  # 对文本进行编码
    input_ids = encoding['input_ids'].squeeze(0)  # 获取input_ids并去除批次维度
    attention_mask = encoding['attention_mask'].squeeze(0)  # 获取attention_mask并去除批次维度

    token_windows = []  # 初始化存储token窗口的列表
    for i in range(0, len(input_ids), stride):  # 按照步长进行滑动窗口
        window_input_ids = input_ids[i:i + max_length]  # 获取当前窗口的input_ids
        window_attention_mask = attention_mask[i:i + max_length]  # 获取当前窗口的attention_mask

        if len(window_input_ids) < max_length:  # 如果当前窗口的长度小于最大长度
            pad_length = max_length - len(window_input_ids)  # 计算需要填充的长度
            window_input_ids = torch.cat([window_input_ids, torch.zeros(pad_length, dtype=torch.long)])  # 填充input_ids
            window_attention_mask = torch.cat([window_attention_mask, torch.zeros(pad_length, dtype=torch.long)])  # 填充attention_mask

        token_windows.append((window_input_ids, window_attention_mask))  # 将当前窗口加入列表

    return token_windows  # 返回所有的token窗口

# 定义自定义数据集类
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_length=128, stride=64):
        self.labels = [labels[label] for label in df['label']]  # 将标签转换为数字
        self.texts = [text for text in df['final']]  # 获取所有的文本
        self.tokenizer = tokenizer  # 保存分词器
        self.max_length = max_length  # 保存最大长度
        self.stride = stride  # 保存步长

    def __len__(self):
        return len(self.labels)  # 返回数据集的长度

    def __getitem__(self, idx):
        text = self.texts[idx]  # 获取指定索引的文本
        label = self.labels[idx]  # 获取指定索引的标签

        token_windows = sliding_window_tokenizer(text, self.tokenizer, self.max_length, self.stride)  # 对文本进行滑动窗口分词
        input_ids_list, attention_mask_list = zip(*token_windows)  # 解压得到input_ids和attention_mask列表

        return {
            'input_ids': torch.stack(input_ids_list),  # 将input_ids堆叠成张量
            'attention_mask': torch.stack(attention_mask_list),  # 将attention_mask堆叠成张量
            'labels': torch.tensor(label, dtype=torch.long)  # 将标签转换为张量
        }

# 定义自定义的collate函数
def custom_collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]  # 获取batch中的所有input_ids
    attention_mask = [item['attention_mask'] for item in batch]  # 获取batch中的所有attention_mask
    labels = torch.tensor([item['labels'] for item in batch], dtype=torch.long)  # 获取batch中的所有标签并转换为张量

    max_num_windows = max([x.size(0) for x in input_ids])  # 获取batch中最大窗口数量
    
    for i in range(len(input_ids)):  # 对batch中的每个样本进行处理
        num_windows = input_ids[i].size(0)  # 获取当前样本的窗口数量
        if num_windows < max_num_windows:  # 如果当前样本的窗口数量小于最大窗口数量
            pad_size = max_num_windows - num_windows  # 计算需要填充的窗口数量
            input_ids[i] = torch.cat([input_ids[i], torch.zeros(pad_size, input_ids[i].size(1), dtype=torch.long)], dim=0)  # 填充input_ids
            attention_mask[i] = torch.cat([attention_mask[i], torch.zeros(pad_size, attention_mask[i].size(1), dtype=torch.long)], dim=0)  # 填充attention_mask

    input_ids = torch.stack(input_ids)  # 将所有input_ids堆叠成一个张量
    attention_mask = torch.stack(attention_mask)  # 将所有attention_mask堆叠成一个张量
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }

# 定义Bert分类器类
class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('model')  # 加载预训练的BertModel
        self.dropout = nn.Dropout(dropout)  # 定义dropout层
        self.linear = nn.Linear(768, 5)  # 定义线性层，输出维度为5

    def forward(self, input_ids, attention_mask):
        batch_size, num_windows, seq_len = input_ids.size()  # 获取batch的大小，窗口数量和序列长度
        input_ids = input_ids.view(-1, seq_len)  # 将input_ids展平
        attention_mask = attention_mask.view(-1, seq_len)  # 将attention_mask展平

        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)  # 获取Bert的输出
        pooled_output = outputs.pooler_output  # 获取池化输出
        pooled_output = pooled_output.view(batch_size, num_windows, -1).mean(dim=1)   # 对窗口进行平均池化

        dropout_output = self.dropout(pooled_output)  # 通过dropout层
        linear_output = self.linear(dropout_output)  # 通过线性层
        return linear_output  # 返回输出

CHECKPOINT_FILE = 'max.pt'  # 定义检查点文件路径

# 定义保存检查点函数
def save_checkpoint(model, optimizer, epoch, val_loss, val_accuracy):
    checkpoint = {
        'model_state_dict': model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(),  # 获取模型状态字典
        'optimizer_state_dict': optimizer.state_dict(),  # 获取优化器状态字典
        'epoch': epoch,  # 保存当前epoch
        'val_loss': val_loss,  # 保存验证损失
        'val_accuracy': val_accuracy  # 保存验证准确率
    }
    torch.save(checkpoint, CHECKPOINT_FILE)  # 保存检查点

# 定义加载检查点函数
def load_checkpoint(model, optimizer):
    if os.path.exists(CHECKPOINT_FILE):  # 检查检查点文件是否存在
        checkpoint = torch.load(CHECKPOINT_FILE)  # 加载检查点
        model_state_dict = checkpoint['model_state_dict']  # 获取模型状态字典
        if isinstance(model, nn.DataParallel):
            model.module.load_state_dict(model_state_dict)  # 加载模型状态字典
        else:
            model.load_state_dict(model_state_dict)
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])  # 加载优化器状态字典
        epoch = checkpoint['epoch']  # 获取保存的epoch
        val_loss = checkpoint['val_loss']  # 获取保存的验证损失
        val_accuracy = checkpoint['val_accuracy']  # 获取保存的验证准确率
        print(f"Checkpoint loaded. Epoch: {epoch}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}")
        return epoch, val_loss, val_accuracy  # 返回加载的状态
    else:
        print("No checkpoint found, starting from scratch.")  # 如果没有检查点文件，则从头开始训练
        return 0, float('inf'), 0.0

# 定义训练函数
def train(model, train_data, val_data, learning_rate, epochs, batch_size):
    train_dataset = Dataset(train_data, tokenizer)  # 创建训练数据集
    val_dataset = Dataset(val_data, tokenizer)  # 创建验证数据集
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)  # 创建训练数据加载器
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, collate_fn=custom_collate_fn)  # 创建验证数据加载器

    use_cuda = torch.cuda.is_available()  # 检查是否可以使用CUDA
    device = torch.device("cuda" if use_cuda else "cpu")  # 设置设备为GPU或CPU
    criterion = nn.CrossEntropyLoss()  # 定义交叉熵损失函数
    optimizer = Adam(model.parameters(), lr=learning_rate)  # 定义Adam优化器

    if use_cuda:
        model = nn.DataParallel(model).cuda()  # 如果使用CUDA，则使用DataParallel并将模型移动到GPU
        criterion = criterion.cuda()  # 将损失函数移动到GPU

    start_epoch, best_val_loss, best_val_accuracy = load_checkpoint(model, optimizer)  # 加载检查点

    for epoch_num in range(start_epoch, epochs):  # 进行训练
        total_acc_train = 0  # 初始化训练准确率
        total_loss_train = 0  # 初始化训练损失

        model.train()  # 将模型设置为训练模式
        for train_input in tqdm(train_dataloader):  # 遍历训练数据
            train_label = train_input['labels'].to(device)  # 获取训练标签并移动到设备
            mask = train_input['attention_mask'].to(device)  # 获取attention_mask并移动到设备
            input_ids = train_input['input_ids'].to(device)  # 获取input_ids并移动到设备

            optimizer.zero_grad()  # 梯度清零
            output = model(input_ids, mask)  # 通过模型获取输出
            batch_loss = criterion(output, train_label)  # 计算批次损失
            total_loss_train += batch_loss.item()  # 累加损失
            acc = (output.argmax(dim=1) == train_label).sum().item()  # 计算准确率
            total_acc_train += acc  # 累加准确率

            batch_loss.backward()  # 反向传播
            optimizer.step()  # 优化器步进

        total_acc_val = 0  # 初始化验证准确率
        total_loss_val = 0  # 初始化验证损失

        model.eval()  # 将模型设置为评估模式
        with torch.no_grad():  # 不进行梯度计算
            for val_input in val_dataloader:  # 遍历验证数据
                val_label = val_input['labels'].to(device)  # 获取验证标签并移动到设备
                mask = val_input['attention_mask'].to(device)  # 获取attention_mask并移动到设备
                input_ids = val_input['input_ids'].to(device)  # 获取input_ids并移动到设备

                output = model(input_ids, mask)  # 通过模型获取输出
                batch_loss = criterion(output, val_label)  # 计算批次损失
                total_loss_val += batch_loss.item()  # 累加损失
                acc = (output.argmax(dim=1) == val_label).sum().item()  # 计算准确率
                total_acc_val += acc  # 累加准确率

        avg_val_loss = total_loss_val / len(val_dataloader)  # 计算平均验证损失
        avg_val_accuracy = total_acc_val / len(val_dataset)  # 计算平均验证准确率

        print(f'''Epochs: {epoch_num + 1} 
              | Train Loss: {total_loss_train / len(train_dataloader): .3f} 
              | Train Accuracy: {total_acc_train / len(train_dataset): .3f} 
              | Val Loss: {avg_val_loss: .3f} 
              | Val Accuracy: {avg_val_accuracy: .3f}''')  # 打印训练和验证结果

        if  1:  # 如果验证损失降低，则保存检查点
            best_val_loss = avg_val_loss  # 更新最好的验证损失
            best_val_accuracy = avg_val_accuracy  # 更新最好的验证准确率
            save_checkpoint(model, optimizer, epoch_num + 1, best_val_loss, best_val_accuracy)  # 保存检查点

# 定义评估函数
def evaluate(model, test_data, batch_size):
    test_dataset = Dataset(test_data, tokenizer)  # 创建测试数据集
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, collate_fn=custom_collate_fn)  # 创建测试数据加载器
    use_cuda = torch.cuda.is_available()  # 检查是否可以使用CUDA
    device = torch.device("cuda" if use_cuda else "cpu")  # 设置设备为GPU或CPU
    if use_cuda:
        model = nn.DataParallel(model).cuda()  # 如果使用CUDA，则使用DataParallel并将模型移动到GPU

    total_acc_test = 0  # 初始化测试准确率
    with torch.no_grad():  # 不进行梯度计算
        for test_input in test_dataloader:  # 遍历测试数据
            test_label = test_input['labels'].to(device)  # 获取测试标签并移动到设备
            mask = test_input['attention_mask'].to(device)  # 获取attention_mask并移动到设备
            input_ids = test_input['input_ids'].to(device)  # 获取input_ids并移动到设备
            output = model(input_ids, mask)  # 通过模型获取输出
            acc = (output.argmax(dim=1) == test_label).sum().item()  # 计算准确率
            total_acc_test += acc  # 累加准确率

    print(f'Test Accuracy: {total_acc_test / len(test_dataset): .3f}')  # 打印测试准确率


In [2]:
df = pd.read_csv('final_data.csv', encoding="GBK")

np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), [int(.8 * len(df)), int(.9 * len(df))])

print(len(df_train), len(df_val), len(df_test))

EPOCHS = 100
model = BertClassifier()
LR = 1e-5
BATCH_SIZE = 1  

train(model, df_train, df_val, LR, EPOCHS, BATCH_SIZE)
evaluate(model, df_test, BATCH_SIZE)

KeyboardInterrupt: 

In [ ]:
import torch  # 导入PyTorch库
import numpy as np  # 导入NumPy库
import pandas as pd  # 导入Pandas库
from transformers import BertTokenizer, BertModel  # 从Transformers库中导入BertTokenizer和BertModel
from torch import nn  # 导入PyTorch的神经网络模块
from tqdm import tqdm  # 导入tqdm库用于进度条显示
import gc  # 导入垃圾回收模块
import os  # 导入操作系统接口模块

# 清理GPU内存
def free_gpu_memory():
    gc.collect()  # 垃圾回收
    torch.cuda.empty_cache()  # 清空CUDA缓存

free_gpu_memory()  # 调用清理GPU内存的函数

tokenizer = BertTokenizer.from_pretrained('model')  # 从预训练模型加载BertTokenizer

# 定义标签字典
labels = {'white': 0, 'sex': 1, 'scam': 2, 'gamble': 3, 'black': 4}
id_to_label = {v: k for k, v in labels.items()}  # 反向标签字典

# 定义滑动窗口分词器函数
def sliding_window_tokenizer(text, tokenizer, max_length=128, stride=64):
    encoding = tokenizer(text, return_tensors='pt', truncation=False)  # 对文本进行编码
    input_ids = encoding['input_ids'].squeeze(0)  # 获取input_ids并去除批次维度
    attention_mask = encoding['attention_mask'].squeeze(0)  # 获取attention_mask并去除批次维度

    token_windows = []  # 初始化存储token窗口的列表
    for i in range(0, len(input_ids), stride):  # 按照步长进行滑动窗口
        window_input_ids = input_ids[i:i + max_length]  # 获取当前窗口的input_ids
        window_attention_mask = attention_mask[i:i + max_length]  # 获取当前窗口的attention_mask

        if len(window_input_ids) < max_length:  # 如果当前窗口的长度小于最大长度
            pad_length = max_length - len(window_input_ids)  # 计算需要填充的长度
            window_input_ids = torch.cat([window_input_ids, torch.zeros(pad_length, dtype=torch.long)])  # 填充input_ids
            window_attention_mask = torch.cat([window_attention_mask, torch.zeros(pad_length, dtype=torch.long)])  # 填充attention_mask

        token_windows.append((window_input_ids, window_attention_mask))  # 将当前窗口加入列表

    return token_windows  # 返回所有的token窗口

# 定义自定义数据集类
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_length=128, stride=64):
        self.labels = [labels[label] for label in df['label']]  # 将标签转换为数字
        self.texts = [text for text in df['final']]  # 获取所有的文本
        self.tokenizer = tokenizer  # 保存分词器
        self.max_length = max_length  # 保存最大长度
        self.stride = stride  # 保存步长

    def __len__(self):
        return len(self.labels)  # 返回数据集的长度

    def __getitem__(self, idx):
        text = self.texts[idx]  # 获取指定索引的文本
        label = self.labels[idx]  # 获取指定索引的标签

        token_windows = sliding_window_tokenizer(text, self.tokenizer, self.max_length, self.stride)  # 对文本进行滑动窗口分词
        input_ids_list, attention_mask_list = zip(*token_windows)  # 解压得到input_ids和attention_mask列表

        return {
            'input_ids': torch.stack(input_ids_list),  # 将input_ids堆叠成张量
            'attention_mask': torch.stack(attention_mask_list),  # 将attention_mask堆叠成张量
            'labels': torch.tensor(label, dtype=torch.long),  # 将标签转换为张量
            'text': text  # 返回原始文本
        }

# 定义自定义的collate函数
def custom_collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]  # 获取batch中的所有input_ids
    attention_mask = [item['attention_mask'] for item in batch]  # 获取batch中的所有attention_mask
    labels = torch.tensor([item['labels'] for item in batch], dtype=torch.long)  # 获取batch中的所有标签并转换为张量

    max_num_windows = max([x.size(0) for x in input_ids])  # 获取batch中最大窗口数量
    
    for i in range(len(input_ids)):  # 对batch中的每个样本进行处理
        num_windows = input_ids[i].size(0)  # 获取当前样本的窗口数量
        if num_windows < max_num_windows:  # 如果当前样本的窗口数量小于最大窗口数量
            pad_size = max_num_windows - num_windows  # 计算需要填充的窗口数量
            input_ids[i] = torch.cat([input_ids[i], torch.zeros(pad_size, input_ids[i].size(1), dtype=torch.long)], dim=0)  # 填充input_ids
            attention_mask[i] = torch.cat([attention_mask[i], torch.zeros(pad_size, attention_mask[i].size(1), dtype=torch.long)], dim=0)  # 填充attention_mask

    input_ids = torch.stack(input_ids)  # 将所有input_ids堆叠成一个张量
    attention_mask = torch.stack(attention_mask)  # 将所有attention_mask堆叠成一个张量
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels,
        'texts': [item['text'] for item in batch]  # 返回原始文本
    }

# 定义Bert分类器类
class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('model')  # 加载预训练的BertModel
        self.dropout = nn.Dropout(dropout)  # 定义dropout层
        self.linear = nn.Linear(768, 5)  # 定义线性层，输出维度为5

    def forward(self, input_ids, attention_mask):
        batch_size, num_windows, seq_len = input_ids.size()  # 获取batch的大小，窗口数量和序列长度
        input_ids = input_ids.view(-1, seq_len)  # 将input_ids展平
        attention_mask = attention_mask.view(-1, seq_len)  # 将attention_mask展平

        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)  # 获取Bert的输出
        pooled_output = outputs.pooler_output  # 获取池化输出
        pooled_output = pooled_output.view(batch_size, num_windows, -1).mean(dim=1)   # 对窗口进行平均池化

        dropout_output = self.dropout(pooled_output)  # 通过dropout层
        linear_output = self.linear(dropout_output)  # 通过线性层
        return linear_output  # 返回输出

CHECKPOINT_FILE = 'model_checkpoint-Copy2.pt'  # 定义检查点文件路径

# 定义加载检查点函数
def load_checkpoint(model):
    if os.path.exists(CHECKPOINT_FILE):  # 检查检查点文件是否存在
        checkpoint = torch.load(CHECKPOINT_FILE)  # 加载检查点
        model_state_dict = checkpoint['model_state_dict']  # 获取模型状态字典
        if isinstance(model, nn.DataParallel):
            model.module.load_state_dict(model_state_dict)  # 加载模型状态字典
        else:
            model.load_state_dict(model_state_dict)
        print(f"Checkpoint loaded.")
    else:
        print("No checkpoint found.")

# 定义评估函数
def evaluate(model, test_data, batch_size):
    test_dataset = Dataset(test_data, tokenizer)  # 创建测试数据集
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, collate_fn=custom_collate_fn)  # 创建测试数据加载器
    use_cuda = torch.cuda.is_available()  # 检查是否可以使用CUDA
    device = torch.device("cuda" if use_cuda else "cpu")  # 设置设备为GPU或CPU
    model.to(device)  # 将模型移动到设备

    if use_cuda:
        model = nn.DataParallel(model).cuda()  # 如果使用CUDA，则使用DataParallel并将模型移动到GPU

    total_acc_test = 0  # 初始化测试准确率

    with torch.no_grad():  # 不进行梯度计算
        for i, test_input in enumerate(tqdm(test_dataloader, desc="Evaluating")):  # 遍历测试数据并显示进度条
            test_label = test_input['labels'].to(device)  # 获取测试标签并移动到设备
            mask = test_input['attention_mask'].to(device)  # 获取attention_mask并移动到设备
            input_ids = test_input['input_ids'].to(device)  # 获取input_ids并移动到设备
            output = model(input_ids, mask)  # 通过模型获取输出
            preds = output.argmax(dim=1)  # 获取预测结果
            acc = (preds == test_label).sum().item()  # 计算准确率
            total_acc_test += acc  # 累加准确率

            # 记录并打印错误的样本
            for idx, pred in enumerate(preds):
                if pred != test_label[idx]:
                    text = test_input['texts'][idx][:100]  # 获取文本的前500个字符
                    true_label = id_to_label[test_label[idx].item()]
                    predicted_label = id_to_label[pred.item()]
                    print(f"True Label: {true_label}, Predicted Label: {predicted_label}, Text: {text}")

    print(f'Test Accuracy: {total_acc_test / len(test_dataset): .3f}')  # 打印测试准确率

df = pd.read_csv('final_data.csv', encoding="GBK")

model = BertClassifier()
BATCH_SIZE = 1

load_checkpoint(model)  # 加载预训练模型
evaluate(model, df, BATCH_SIZE)  # 进行评估


Checkpoint loaded.


Evaluating:   4%|▍         | 99/2220 [00:24<08:57,  3.95it/s]

True Label: black, Predicted Label: white, Text: 欣然业务网 POST NOTIFICATIONS WAKE LOCK ACCOUNTS coloros mcs RECIEVE MCS MESSAGE chuangdun borderkeeper A


Evaluating:   9%|▉         | 208/2220 [00:49<04:45,  7.04it/s]

True Label: black, Predicted Label: white, Text: 唐十八 CHANGE NETWORK STATE WAKE LOCK FLASHLIGHT WRITE SETTINGS vivo notification BADGE ICON CAPTURE AU


...

Evaluating:  87%|████████▋ | 1924/2220 [07:23<01:17,  3.81it/s]

True Label: sex, Predicted Label: gamble, Text: 在阴暗的房间中和独孤的女孩子的物语 get_at centre body_politic BLUETOOTH internet dev zzksoft A META appcompat app RSA


Evaluating:  87%|████████▋ | 1941/2220 [07:28<01:13,  3.81it/s]

True Label: white, Predicted Label: sex, Text: 胡桃大战史莱姆 modify web shit Wake_Island lock query all parcel project compose outside z modify val Unite


Evaluating:  88%|████████▊ | 1955/2220 [07:31<01:04,  4.14it/s]